# Single Cell Analysis in R 2023 Notebook 1

This workbook presents the first part of a basic work flow for scRNAseq analysis
1. Loading data
2. Quality Control
   2.1 Cell filtering
   2.2 Doublet assessment
3. Normalization
4. Feature selection
5. Dimensionality reduction

Example data set description:



# Analysis workflow

Load your required libraries.
You need to have these libraries installed. 


In [ ]:
Sys.time()
library(Seurat)
library(tidyverse)
library(DoubletFinder)


Load the data: Starting with the adolescent data.



In [ ]:
# load data

# insert the pathway to the location of the data
setwd("C:/Users/Home/Documents/GitHub/SingleCell_Workshop/")

adolescent_data <- Read10X("data/Adolescent_14_YO_raw_feature_bc_matrix/", 
                      strip.suffix = TRUE )

#Look at the dimensions of the matrix
dim(adolescent_data)

#Look at a small part of the data
adolescent_data[1:5, 1:5]

#Look at the distribution of the number of UMIs per cell
colSums(adolescent_data) %>% summary
#Look at the distribution of the number of genes per cell  
colSums(adolescent_data > 0) %>% summary


Remove barcodes with too few genes that could be empty droplets



In [ ]:
#Remove barcodes with less than 200 genes detected (you can select a different value here)
#You could use something more refined here, like EmptyDrops
adolescent_data <- adolescent_data[, colSums(adolescent_data > 0)> 200]
dim(adolescent_data)


Now we have gone from 6794880 barcodes to 7371 barcodes. These barcodes now represent cells. 



Filter genes and create a Seurat object


In [ ]:
#We might not want to include genes that occur in few cells are no cells.  Here we will filter out genes/transcripts that are in less than 3 cells. 

#Make a Seurat object
#Removing any genes detected in less than 3 cells
adolescent_data_seurat <- CreateSeuratObject(adolescent_data, project = "Adolescent", min.cells = 3)
# look at the object dimensions
adolescent_data_seurat


We have now gone from 33538 RNA transcripts/genes to 19866 genes.


Now we will look at some metadata in the seurat object


In [ ]:
#Look at some metadata
adolescent_data_seurat@meta.data %>% names
# there are the meta data we currently have in our seurat object



Data distribution


In [ ]:
# look at the distribution of total counts of RNA across cells
adolescent_data_seurat$nCount_RNA %>% summary

# look at the distribution of unique RNA transcripts across cells

adolescent_data_seurat$nFeature_RNA %>% summary


Visualize the distributions



In [ ]:
VlnPlot(adolescent_data_seurat, features = c("nCount_RNA","nFeature_RNA"), pt.size = 0)



Repeat all these steps with the adult data. 



In [ ]:
#Repeat above steps for the adult dataset, use a minimum genes per cell barcode cutoff of 200
#With real data you would use the same cutoff across samples 
#You would decide the cutoff based on what works best across your samples. 
#Can be left as an exercise i.e. no code provided until after the workshop or code provided in a different file

adult_data <- Read10X(data.dir = "data/Adult_41_YO_raw_feature_bc_matrix/", 
                    strip.suffix = TRUE)
dim(adult_data)
adult_data[1:5, 1:5]
colSums(adult_data) %>% summary
colSums(adult_data > 0) %>% summary
adult_data <- adult_data[, colSums(adult_data > 0)> 200]
dim(adult_data)
adult_data_seurat <- CreateSeuratObject(adult_data, project = "Adult", min.cells = 3)
adult_data_seurat
adult_data_seurat@meta.data %>% names
adult_data_seurat$nCount_RNA %>% summary


Now we want to add some sample meta data



In [ ]:
#Make a tibble with the age info
colnames(adolescent_data_seurat) %>% head
adolescent_sample_metadata <- tibble(Cell_barcodes = colnames(adolescent_data_seurat), 
                                age = rep(14, dim(adolescent_data_seurat)[2]))
head(adolescent_sample_metadata)

# this sample is from one brain and the age is 14 years so we will add that data for all the cells


The cells have previously been annotated - we will add in those annotations



In [ ]:
# input file path

input_file <- paste("data/HA799_14YO_metadata.csv",sep="")

#Read in metadata file with original cluster info
adolescent_sample_clusters <- read_csv(file = input_file)
head(adolescent_sample_clusters)



Select the columns we need


In [ ]:
adolescent_sample_clusters <- adolescent_sample_clusters %>% select(-c(2:7))

colnames(adolescent_sample_clusters) <- c("Cell_barcodes","Original_clusterID")

head(adolescent_sample_clusters)



Now we will combine the two meta data ojbects and add the meta data into the Seurat object


In [ ]:
#Combine the age and original cluster metadata into a single object
#Note that not all the cells we have retained were present in the original analysis, we will keep all our called cells
adolescent_sample_metadata %>% left_join(adolescent_sample_clusters, by = "Cell_barcodes") %>% 
  column_to_rownames("Cell_barcodes") %>% as.data.frame -> adolescent_sample_metadata
adolescent_sample_metadata %>% head 

#Here we add the metadata to our Seurat object 
adolescent_data_seurat <- AddMetaData(adolescent_data_seurat, metadata = adolescent_sample_metadata)
adolescent_data_seurat@meta.data %>% head


In [ ]:
#Remove the original matrix, and other unnecessary objects to clean up space 
rm(adolescent_data, adolescent_sample_metadata, adolescent_sample_clusters)



Repeat adding meta data for the adult object


In [ ]:
#Repeat the process of adding metadata for the adult sample
input_file <- paste("data/HA801_41YO_metadata.csv",sep="")

adult_sample_metadata <- tibble(CellName = colnames(adult_data_seurat), 
                                age = rep(41, dim(adult_data_seurat)[2]))
adult_sample_clusters <- read_csv(file = input_file) %>% select(-c(2:7))
colnames(adult_sample_clusters) <- c("CellName", "OriginalCluster")
adult_sample_metadata %>% full_join(adult_sample_clusters, by = "CellName") %>% 
  column_to_rownames("CellName") %>% as.data.frame -> adult_sample_metadata
adult_data_seurat <- AddMetaData(adult_data_seurat, metadata = adult_sample_metadata)
adult_data_seurat@meta.data %>% head
# clean up extra files
rm(adult_data, adult_sample_clusters, adult_sample_metadata)


Filter out unwanted cells



In [ ]:
# Example cell filtering based on mitochondrial count percentage and number of UMIs ----------

#Calculate the percentage of mitochondrially encoded mitochondrial genes
adolescent_data_seurat <- PercentageFeatureSet(adolescent_data_seurat, pattern = "^MT-", col.name = "percent.MT")
adolescent_data_seurat$percent.MT %>% summary



Visualize the mitochondria


In [ ]:
VlnPlot(adolescent_data_seurat, features = "percent.MT", pt.size = 0.001)

#Remove any cells with more than 20% mitochondrial counts
adolescent_data_seurat <- subset(adolescent_data_seurat, percent.MT < 20)

#Remove cells with very high UMI counts, which may be possible multiplets
adolescent_data_seurat <- subset(adolescent_data_seurat, nCount_RNA < 20000)

# see the results
VlnPlot(adolescent_data_seurat, features = c("percent.MT", "nCount_RNA", "nFeature_RNA"), pt.size = 0.001)


Repeat filtering for the adult sample



In [ ]:
#Repeat filtering based on mitochondrial genes and number of UMIs for the adult sample
#Use the same criteria as we used for the adolescent sample
#Can be left as an exercise
adult_data_seurat <- PercentageFeatureSet(adult_data_seurat, pattern = "^MT-", col.name = "percent.MT")
adult_data_seurat$percent.MT %>% summary
adult_data_seurat <- subset(adult_data_seurat, percent.MT < 20)
VlnPlot(adult_data_seurat, features = c("percent.MT", "nCount_RNA", "nFeature_RNA"))
adult_data_seurat <- subset(adult_data_seurat, nCount_RNA < 20000)
VlnPlot(adult_data_seurat, features = c("percent.MT", "nCount_RNA", "nFeature_RNA"))



Cell cycle scoring (optional)


In [ ]:
adolescent_data_seurat <- CellCycleScoring(adolescent_data_seurat, g2m.features = cc.genes$g2m.genes, s.features = cc.genes$s.genes)
VlnPlot(adolescent_data_seurat, features = c("S.Score", "G2M.Score"))

adult_data_seurat <- CellCycleScoring(adult_data_seurat, g2m.features = cc.genes$g2m.genes, s.features = cc.genes$s.genes)
VlnPlot(adult_data_seurat, features = c("S.Score", "G2M.Score"))


Data normalization


In [ ]:
# Normalize data (log normalization) and select genes with variable expression across cells --------------------------------------

adolescent_data_seurat <- NormalizeData(adolescent_data_seurat, normalization.method = "LogNormalize", scale.factor = 10000)

#Check out the effect of normalization
GetAssayData(adolescent_data_seurat, assay = "RNA", slot = "data") %>% expm1 %>% colSums %>% head
GetAssayData(adolescent_data_seurat, assay = "RNA", slot = "counts") %>% colSums %>% head


Finding Variable feature with two different methods


In [ ]:
# Dispersion
#Find and plot variable features (in our case genes) with dispersion based method
adolescent_data_seurat <- FindVariableFeatures(adolescent_data_seurat, selection.method = "disp", nfeatures = 2000)
VariableFeaturePlot(adolescent_data_seurat)
disp_var  <- VariableFeatures(adolescent_data_seurat)


In [ ]:
# VST
#Find and plot variable features (in our case genes) with vst based method
adolescent_data_seurat <- FindVariableFeatures(adolescent_data_seurat, selection.method = "vst", nfeatures = 2000)

VariableFeaturePlot(adolescent_data_seurat)



Compare both


In [ ]:
#Compare the two methods, look at some of the variable genes
intersect(disp_var, VariableFeatures(adolescent_data_seurat)) %>% length
VariableFeatures(adolescent_data_seurat) %>% head(n = 20)



Repeat for the adult sample


In [ ]:
#Repeat normalization and variable feature selection for adult sample
#Use method vst and 2000 variable features 
#Can be left as an exercise 

adult_data_seurat <- NormalizeData(adult_data_seurat, normalization.method = "LogNormalize", scale.factor = 10000)
adult_data_seurat <- FindVariableFeatures(adult_data_seurat, selection.method = "vst", nfeatures = 2000)
VariableFeaturePlot(adult_data_seurat)
VariableFeatures(adult_data_seurat) %>% head(n = 20)


Dimensionality reduction PCA and UMAP



In [ ]:
#Scaling is recommended before PCA, as otherwise highly expressed genes will have a disproportionate effect
adolescent_data_seurat <- ScaleData(adolescent_data_seurat, vars.to.regress = "percent.MT")
adolescent_data_seurat@assays$RNA@scale.data %>% dim

#Linear dimensionality reduction
#Choosing the number of PCs can depend on how many cells you have
adolescent_data_seurat <- RunPCA(adolescent_data_seurat, assay = "RNA", npcs = 30)
#PCAPlot(adolescent_data_seurat, group.by = "Original_clusterID")
# this a confusing because we won't normally have this 

PCAPlot(adolescent_data_seurat)

#Assess how many PCs capture most of the information in the data 
ElbowPlot(adolescent_data_seurat, ndims = 30)


In [ ]:
# Jackstraw

#Assess how many PCs capture most of the information in the data 
adolescent_data_seurat <-JackStraw(adolescent_data_seurat, reduction = "pca",
  dims = 30)
adolescent_data_seurat <- ScoreJackStraw(adolescent_data_seurat, reduction = "pca", dims = 1:30)
JackStrawPlot(adolescent_data_seurat, dims = 1:30)



Non-linear dimensional reduction using UMAP


In [ ]:
#Non-linear dimensionality reduction
#Choosing how many PCs to input can depend on the elbow plot and on the number of cells
#There are many parameters that can e tweaked and optimized in a UMAP plot
#You can see some demos here: https://pair-code.github.io/understanding-umap/
adolescent_data_seurat <- RunUMAP(adolescent_data_seurat, dims = 1:10)
UMAPPlot(adolescent_data_seurat)



Repeat dimensional reduction with the adult sample


In [ ]:
#Repeat dimensional reduction for adult sample, use 10 PCs for 
adult_data_seurat <- ScaleData(adult_data_seurat, vars.to.regress = "percent.MT")
adult_data_seurat <- RunPCA(adult_data_seurat, assay = "RNA", npcs = 30)
PCAPlot(adult_data_seurat)
ElbowPlot(adult_data_seurat)
adult_data_seurat <- RunUMAP(adult_data_seurat, dims = 1:10)
UMAPPlot(adult_data_seurat)


Doublet identification



In [ ]:
# Assess possible doublets  -----------------------------------------------
#Using instructions here: https://github.com/chris-mcginnis-ucsf/

#First we have to find a pK which determines how big of a neighborhood will be examined for doublets
#This should be chosen for each library separately
#First we test a number of pN (proportion of generated artificial doublets) and pK
#We get different lists of probabilities of artifical nearest neighbors with these tested parameters
#Also keep in mind the results are not deterministic (every run will give slightly different results)
sweep.res.list_adolescent <- paramSweep_v3(adolescent_data_seurat, PCs = 1:15, sct = FALSE)

#We do not have the "ground truth" regarding doublets, such from from genotype data for pooled samples 
#We sumamrize the performance of the range of pN=pK parameters we tested
sweep.stats_adolescent <- summarizeSweep(sweep.res.list_adolescent, GT = FALSE)

#Here the "best" pK for the data is chosen based on a metric determined by the DoubletFinder developers
#Which performs best in datasets where the ground truth is known
bcmvn_adolescent <- find.pK(sweep.stats_adolescent)
ggplot(bcmvn_adolescent, aes(x = pK, y = BCmetric, group = "Sweep")) + geom_point() + geom_line() + 
  theme(axis.text.x = element_text(angle = 90))

#We will pick pK = 0.29
#We are not going to use our clustering information to estimate "homotypic" doublets
#We are simply going to use an expected doublet formation rate of 7.5%
nExp_poi <- round(0.075*nrow(adolescent_data_seurat@meta.data))
adolescent_data_seurat <- doubletFinder_v3(adolescent_data_seurat, PCs = 1:15, pN = 0.25, pK = 0.09, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)

#Here we update the Seurat object version just in case the one returned by DoubletFinder is an older version
adolescent_data_seurat <- UpdateSeuratObject(adolescent_data_seurat)

#Visualize and assess the cells called as probable doublets
UMAPPlot(adolescent_data_seurat, group.by = "DF.classifications_0.25_0.09_533")

# table of doublets and signlets
adolescent_data_seurat$DF.classifications_0.25_0.09_533 %>% table

# visualize the features in doublets and singlets
VlnPlot(adolescent_data_seurat, features = c("nCount_RNA", "nFeature_RNA", "percent.MT", "pANN_0.25_0.09_533"), 
        group.by = "DF.classifications_0.25_0.09_533", pt.size = 0.001)


Repeat doublet detection in adult sample



In [ ]:
#Repeat the above analysis with the adult sample
sweep.res.list_adult <- paramSweep_v3(adult_data_seurat, PCs = 1:10, sct = FALSE)
sweep.stats_adult <- summarizeSweep(sweep.res.list_adult, GT = FALSE)
bcmvn_adult <- find.pK(sweep.stats_adult)
ggplot(bcmvn_adult, aes(x = pK, y = BCmetric, group = "Sweep")) + geom_point() + geom_line() +
  theme(axis.text.x = element_text(angle = 90))
nExp_poi <- round(0.075*nrow(adult_data_seurat@meta.data))
adult_data_seurat <- doubletFinder_v3(adult_data_seurat, PCs = 1:10, pN = 0.25, pK = 0.17, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)
adult_data_seurat <- UpdateSeuratObject(adult_data_seurat)

UMAPPlot(adult_data_seurat, group.by = "DF.classifications_0.25_0.17_146")
adult_data_seurat$DF.classifications_0.25_0.17_146 %>% table
VlnPlot(adult_data_seurat, features = c("nCount_RNA", "nFeature_RNA", "percent.MT", "pANN_0.25_0.187_146"), 
        group.by = "DF.classifications_0.25_0.17_146", pt.size = 0.001)


Save the data objects for later



In [ ]:
saveRDS(adolescent_data_seurat, "adolsecentSeurate.Rds")
saveRDS(adult_data_seurat, "adultSeurate.Rds")
Sys.time()
